In [1]:
import pandas as pd 
import numpy as np 
import os

In [2]:
#read in the hsp.tsv file
path = '/home/jake/Downloads/Salmo_Project/'
hsp = os.listdir(f"{path}heatshock_proteins")
not_hsps_sample = os.listdir(f"{path}/not_heatshock_proteins_control_group")
Orthogroups = f'{path}Orthogroups/'
Orthogroups = pd.read_csv(f'{Orthogroups}Orthogroups.tsv', sep = '\t', index_col=0)

In [3]:
#Need to add the species name to all of the fasta headers :(

def parse_fasta_to_df(file):
    fasta_list = []
    current_header = None
    current_sequence = ""
    with open(file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith(">"):
                if current_header:
                    fasta_list.append([current_header, current_sequence])
                current_header = line[1:]
                current_sequence = ""
            else:
                current_sequence += line.strip()
    return pd.DataFrame(fasta_list, columns=['header', 'sequence'])


def fix_headers(df):
    df['new_head'] = df['Species'] + "_" + df['header']
# Make a new data frame that has only the new_head and sequence columns
    df = df[['new_head', 'sequence']]
    with open('/home/jake/Downloads/OG0000151_modified.fasta', 'w') as f:
        for i, row in dfx.iterrows():
            header = row['new_head']
            sequence = row['sequence']
            f.write(f">{header}\n{sequence}\n")


In [4]:
#make a filtered dataframe of orthogroups that has the column names of orthogroups and only one row that does not have any Nan values
filtered = Orthogroups.dropna(axis=1, how='all')
filtered = filtered.dropna(axis=0, how='any')
#Keep only the headers and the first row
filtered = filtered.iloc[0:1, 0:]
#Drop the values after the first ,
filtered = filtered.applymap(lambda x: x.split(',')[0])
filtered = filtered.applymap(lambda x: x.split('G0')[0])

In [5]:
#drop the index
filtered = filtered.reset_index(drop=True)
#createa a dicionary with colnames as key: and the values as the orthogroup
filtered_dict = filtered.to_dict('records')[0]
filtered_dict

{'Danio_rerio': 'ENSDAR',
 'Hucho_hucho': 'ENSHHU',
 'Oncorhynchus_kisutch': 'ENSOKI',
 'Oncorhynchus_mykiss': 'ENSOMY',
 'Oncorhynchus_tshawytscha': 'ENSOTS',
 'Salmo_salar': 'ENSSSA',
 'Salmo_trutta': 'ENSSTU'}

In [6]:
folderpath = '/home/jake/Downloads/Salmo_Project/heatshock_proteins/'

# Loop through every file in the folder
for filename in os.listdir(folderpath):
    # If the file is a fasta file, process it
    if filename.endswith('.fa'):
        # Open the file and read its contents
        with open(os.path.join(folderpath, filename), 'r') as f:
            lines = f.readlines()
        # Loop through every line in the file
        for i in range(len(lines)):
            line = lines[i].strip()
            # If the line is a header, process it
            if line.startswith('>'):
                # Check if the header starts with any value in the filtered_dict
                for key, value in filtered_dict.items():
                    if value in line:
                        # If it does, append the corresponding key to the start of the header with an underscore
                        new_header = f'>{key}_{line[1:]}'
                        # Replace the old header with the new header in the lines list
                        lines[i] = new_header + '\n'
                        break
        # Write the modified contents back to the file
        with open(os.path.join(folderpath, filename), 'w') as f:
            f.writelines(lines)


    